# Simulated Dataset

In this notebook you can find code snippets that I always reuse for simulating dataset

- toc: true 
- badges: true
- comments: false
- categories: [r]


*TOC*
* writing a function for creating a dataset with a desired number of rows and cols given a mean and an sd (same for all cols)
* writing a function for creating a small dataset (n col < 5) with a desired number of rows and cols given a mean and an sd (different for each cols)
* writing a function for creating a dataset (n col > 5) with a desired number of rows and cols given a mean and an sd (different for each cols)
* writing functions for creating automatic labels for ID and categories

In [1]:

#°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°
#  loading required libraries for this notebook
#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°

#loading libraries

library(ggplot2)
library(gridExtra)
library(data.table)

#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°
#  Example 1 a very simple test dataset
#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°
# we are creating a dataframe from a matrix obtained replicating x 
# the desired number_of_cols a vector of length number_of_rows from 
# a normal distribution rnorm with a mean of my mean and standard deviation as sd

number_of_rows <- 7
number_of_cols <- 6
my_mean <- 2
my_sd <- 0.5

newdat <-  as.data.frame( 
           replicate(
           number_of_cols,
           rnorm(n = number_of_rows, mean = my_mean, sd = my_sd ))
           )



#in order to print a fancy table
newdat


V1,V2,V3,V4,V5,V6
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.179453,1.367333,2.310614,1.894637,1.774971,2.381914
1.030141,1.502799,2.205031,2.036801,2.419301,1.203293
2.374548,1.195058,3.186200,1.465399,2.234618,1.988530
3.059161,1.574156,1.524985,2.813610,2.610743,2.212044
1.583258,2.177160,1.261412,1.966853,1.949342,1.250590
2.124564,2.713874,2.709232,2.776729,1.871581,1.786535
1.828765,2.327800,1.742169,1.812997,1.626216,1.063126


-  Code for creating small (n col < 5) dataset.
Each column has its own mean and sd. 
In the example reported we have n = 3  (A, B, C)
with n row = 50. means are 100,110,120 and sd 1,2,3


In [2]:

#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°
#  Ex 2 Another way for a simple dataset
#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°

#in order to obtain ALWAYS same "random" results REMEMBER TO initialize the seed 
#set.seed(42) 

number_of_rows <- 5
A <- rnorm( n=number_of_rows, mean=100, sd=1 ) 
B <- rnorm( n=number_of_rows, mean=110, sd=2 ) 
C <- rnorm( n=number_of_rows, mean=120, sd=2 ) 

dat=data.frame(A,B,C) 

dat



A,B,C
<dbl>,<dbl>,<dbl>
99.77124,111.6332,117.2913
98.76795,111.9685,118.6367
101.41129,107.3180,118.2394
100.97150,111.7011,119.4043
99.68483,109.9691,121.6785


- same as the one above but more useful for dataset with i columns n col > 5

In [3]:

#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°
#  Ex 3 recipes for adding labels
#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°

number_of_rows = 3
means=c(100, 120, 130, 145)
sds=c(10 ,20 ,40 ,10)

dat <- lapply(
       seq(1,length(means)) ,
       function(x) rnorm(number_of_rows,m = means[x], sds[x])
       )
dat <- as.data.frame(do.call(cbind, dat))
 
names_length = 3
dictionary_size <- 10
my_labels <- sort(
            replicate(
            length(means),
            paste(sample(LETTERS[1:dictionary_size],
            names_length,
            replace = TRUE),
            collapse="")
            )
            )
my_labels <- unlist(strsplit(my_labels," "))  
colnames(dat) <- my_labels

dat

AJE,DJI,EHI,HHI
<dbl>,<dbl>,<dbl>,<dbl>
87.51604,99.23154,129.71654,156.1370
99.91220,95.44262,31.42456,138.0569
84.50430,125.61206,173.80734,151.2865


- Same as above but shorter 

In [4]:

#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°
#  Ex 4 a variation on recipe 2 
#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°

# building a function for generating data with custom number of rows, means and sds

simpleDataset <- function(number_of_rows,means,sds)
{
l <- length(means)
res <- lapply(seq(1:l),function(x) 
	   eval(
	   parse(text=paste("rnorm(",number_of_rows,",",means[x],",",sds[x],")",sep="")))
	   ) 
dat <- data.frame((sapply(res,c)))
id <- rownames(dat)
dat <-  cbind(id=id,dat)
dt <- data.table(dat)
return(dt)
}


dat1 <- simpleDataset(number_of_rows=30,
					  means=c(180,200,205),
				      sds=c(30,20,25))
				  
dat2 <- simpleDataset(number_of_rows=30,
					  means=c(45,50,35),
				      sds=c(2,10,5))
					  
dat <-  rbind(dat1,dat2)
# rearranging table using melt from data.table 
dt.melt <- melt(dat, id.vars="id")
colnames(dt.melt) <- c("id","category","var1")


- to create sample names or labels (see https://stackoverflow.com/a/60789938/6483091)


In [5]:

#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°
#  More recipes for labelling
#+++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°+++++++++°

#short names
my_labels <- letters[1:5]
my_labels
# or <- 
my_labels <- LETTERS[1:5]
my_labels
# or arbitrary number of letters using roman letters as in the 
#function 
# letters() or LETTERS()

dictionary_size <- 7
label_length <- 5 
n_replicates <- 3

#random

my_labels <- replicate(
             n_replicates,
             paste(sample(LETTERS[1:dictionary_size],
             label_length,
             replace = TRUE),
             collapse="")
             )
my_labels

#sorted

my_labels_sorted <- sort(replicate(
                    n_replicates, 
                    paste
                    (sample(LETTERS[1:dictionary_size],
                     label_length,
                     replace = TRUE),
                     collapse="")
                    )
                    )
my_labels_sorted
#if you want to mix letters and numbers

alfanum_labels <- paste0(rep(LETTERS[1:dictionary_size],
                             each = n_replicates),
                             sep = "-",
                             1:n_replicates)
alfanum_labels

[1] "a" "b" "c" "d" "e"

[1] "A" "B" "C" "D" "E"

[1] "BGGFC" "GCCBC" "BACDC"

[1] "AFEGF" "EBCCG" "GEEGF"

[1] "A-1" "A-2" "A-3" "B-1" "B-2" "B-3" "C-1" "C-2" "C-3" "D-1" "D-2" "D-3"
[13] "E-1" "E-2" "E-3" "F-1" "F-2" "F-3" "G-1" "G-2" "G-3"